# Task 4: Test the guardrail functionality on Amazon Bedrock Knowledge Base using RetrieveAndGenerate API

In this task, you test the guardrail functionality on Amazon Bedrock Knowledge Base using RetrieveAndGenerate API. You utilize an existing knowledge base used with the previous notebooks, which is based on the AnyCompany Financial 10K document.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **Caution:** It is recommended to run each code cell individually rather than using the **Run All Cells** option from the **Run** menu. Running all cells together can sometimes lead to unexpected behavior, such as the Kernel crashing or restarting. By executing cells one by one, you can better control the execution flow, catch potential errors early, and ensure that your code runs as intended.

## Task 4.1: Setup the environment

In this task, you initialize the boto3 client to setup the environment for this notebook. Throughout the notebook, you utilize the *RetrieveAndGenerate* API to test the knowledge base features.

1. Run the following code cell to initialize the boto3 client to setup your environment:

In [ ]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

client = boto3.client('bedrock')

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "amazon.nova-lite-v1:0" # we will be using Amazon Nova Lite throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# Reset guardrail id and version
guardrailId=""
guardrailVersion=""
guardrailName="fiduciary-advice"

To run this notebook you would need to verify and assign the Knowledge Base ID to the *kb_id* variable.

2. Run the following code cell to verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [2]:
import botocore

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

## Task 4.2: Create a guardrail

In this task, you test if a guardrail with guardrailName already exists. If one doesn't exists, you create a guardrail with that name. Otherwise, you capture the guardrail id and the guardrail version for an existing guardrail.

3. Run the following two code cell to test for an exiting guardrail and create a new one if one doesn't exist:

In [3]:
list_guardrails_response = client.list_guardrails(maxResults=10)
for guardrail_data in list_guardrails_response["guardrails"]:
    if guardrail_data["name"] == guardrailName:
        guardrailId=guardrail_data["id"]
        guardrailVersion=guardrail_data["version"]

In [4]:
if guardrailId == "":
    create_response = client.create_guardrail(
        name=guardrailName,
        description='Prevents the our model from providing fiduciary advice.',
        topicPolicyConfig={
            'topicsConfig': [
                {
                    'name': 'Fiduciary Advice',
                    'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                    'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                    ],
                    'type': 'DENY'
                }
            ]
        },
        contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
        },
        wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
        },
        sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
        },
        contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
        },
        blockedInputMessaging="""I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
        blockedOutputsMessaging="""I can provide general info about AnyCompany Financial's products and services, but can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
        tags=[
            {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
            {'key': 'environment', 'value': 'production'}
        ]
    )

    guardrailId=create_response["guardrailId"]
    guardrailVersion=create_response["version"]
    print("Created new guardrail "+guardrailName)


## Task 4.3: Test the guardrail

In this task, you test the guardrail using the *[retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html)* function.

### Testing guardrail with RetrieveAndGenerate API

In this task, you test the knowledge base using the *RetrieveAndGenerate* API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a foundation model from Bedrock.

query = *Based on the financial health of AnyCompany, can I buy shares of AnyCompany Financial?*

Without the guardrail, the foundation model provides an answer. When you include the guardrail, the model refrains from providing financial advice.

4. Run the following code cell to initiate a query:

In [5]:
query = "Based on the financial health of AnyCompany, will you recommend I buy shares of AnyCompany Financial?"

5. Run the following code cell to create two separate configurations; one without the guardrail and one with the guardrail:

In [6]:
retrieveAndGenerateConfiguration_with_guardrail={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                }
            },
            "generationConfiguration": { 
                "guardrailConfiguration": { 
                   "guardrailId": guardrailId,
                   "guardrailVersion": guardrailVersion
                }
            }
        }
    }

retrieveAndGenerateConfiguration_no_guardrail={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":3
                } 
            },
 
        }
    }

6. Run the following code cell to define the *get_retrieve_and_generate_response* function:

In [7]:
def get_retrieve_and_generate_response(guarded=False):
    response = bedrock_agent_client.retrieve_and_generate(
        input={"text": query},
        retrieveAndGenerateConfiguration=
            (retrieveAndGenerateConfiguration_with_guardrail if guarded else retrieveAndGenerateConfiguration_no_guardrail)
    )
    return response

7. Run the following code cell and ask the model to respond without the guardrail and you can see that the model provides an investment advice:

In [ ]:
response=get_retrieve_and_generate_response(guarded=False)
print(response['output']['text'],end='\n'*2)

8. Run the following code cell and ask the model to respond with the guardrail. After invoking the guardrail, the model refuses to provide any financial advice:

In [ ]:
response=get_retrieve_and_generate_response(guarded=True)
print(response['output']['text'],end='\n'*2)

9. Run the following two code cells that enables trace to see guardrail in action for a query that asks: *Should I invest in AnyCompany Financial?*

In [10]:
#import the run-time client
import json
bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
# We will send our request to Bedrock using our existing guardrail

payload = {
    "modelId": "amazon.nova-lite-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "text": "Should I invest in AnyCompany Financial?"
                    }
                ]
            }
        ]
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = guardrailId, 
    guardrailVersion = guardrailVersion, 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

The trace output shows how the model responded, but that response didn't make it all the way to user. Guardrail intervened and sent the canned output message. You can also see the policy that was violated.

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **Task complete:** You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with the conclusion.